# Jet Compression using an Autoencoder

In [1]:
import sys
BIN = '../../'
sys.path.append(BIN)
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset

from my_nn_modules import AE_big, get_data, fit

import my_matplotlib_style as ms
mpl.rc_file(BIN + 'my_matplotlib_rcparams')

In [2]:
force_cpu = False

if force_cpu:
    device = torch.device('cpu')
else:
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
print('Using device:', device)

Using device: cpu


## Loading and preparing the data

In [3]:
# Load data
train = pd.read_pickle(BIN + 'processed_data/train.pkl')
test = pd.read_pickle(BIN + 'processed_data/test.pkl')

train.head(10)

,pT,eta,phi,E
1367643,300.752869,0.118391,-1.460827,303.719818
1739612,79.819145,4.535507,-1.185196,3722.832764
546066,220.472305,1.314261,-0.943992,440.195190
213212,74.533775,0.010658,-0.424795,74.770332
150544,220.668121,2.432910,-2.308348,1266.681030
574837,159.841782,0.731125,-1.150598,207.133514
1171687,306.125305,0.702473,0.437615,385.713013
1608254,189.011673,-1.387443,0.490634,403.126709
1618830,194.996719,-1.022815,-1.560437,306.534515
637947,186.887146,-0.621282,-0.465523,226.002701


## Normalizing and splitting into train and test datasets

In [4]:
np.max(train)

pT      20721.585938
eta         4.785440
phi         3.141583
E      146259.562500
dtype: float32

In [5]:
train.mean()

pT     226.349228
eta     -0.021712
phi      0.008840
E      596.682495
dtype: float32

In [6]:
# Normalize
train_mean = train.mean()
train_std = train.std()
#train_max = np.max(train)

train = (train - train_mean) / train_std
#train = (train - train_mean) / train_max
test = (test - train_mean) / train_std
#test = (test - train_mean) / train_max

train_x = train
test_x = test
train_y = train_x  # y = x since we are building and AE
test_y = test_x

train_ds = TensorDataset(torch.tensor(train_x.values), torch.tensor(train_y.values))
valid_ds = TensorDataset(torch.tensor(test_x.values), torch.tensor(test_y.values))

## Training

In [7]:
bs = 256  # batch size
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
loss_func = nn.MSELoss()

In [8]:
n_features = len(train.loc[0])
model_big = AE_big(n_features=n_features)

In [9]:
opt = optim.Adam(model_big.parameters(), lr=1e-3)
epochs = 10
summary_df = fit(epochs, model_big, loss_func, opt, train_dl, valid_dl, device)

Epoch: 0 Train Loss: 3.722e-01 Val Loss: 2.519e-01, Time: 0:01:01.558559
Epoch: 1 Train Loss: 6.817e-02 Val Loss: 3.217e-03, Time: 0:00:57.149382
Epoch: 2 Train Loss: 2.111e-02 Val Loss: 1.507e-03, Time: 0:00:56.414266
Epoch: 3 Train Loss: 2.009e-02 Val Loss: 1.050e-03, Time: 0:01:00.674522
Epoch: 4 Train Loss: 1.968e-02 Val Loss: 8.967e-04, Time: 0:01:05.030975
Epoch: 5 Train Loss: 1.938e-02 Val Loss: 7.624e-04, Time: 0:01:01.705871
Epoch: 6 Train Loss: 1.916e-02 Val Loss: 1.014e-03, Time: 0:00:57.593481
Epoch: 7 Train Loss: 1.897e-02 Val Loss: 5.474e-04, Time: 0:01:01.224138
Epoch: 8 Train Loss: 1.881e-02 Val Loss: 5.205e-04, Time: 0:01:06.050725
Epoch: 9 Train Loss: 1.864e-02 Val Loss: 6.158e-04, Time: 0:01:10.818058
Training complete in 10m 18s


In [10]:
summary_df

,Epoch,val_loss,train_loss,epoch_time
0,0,0.251915,0.372206,70.818058
1,1,0.003217,0.068170,70.818058
2,2,0.001507,0.021113,70.818058
3,3,0.001050,0.020092,70.818058
4,4,0.000897,0.019679,70.818058
5,5,0.000762,0.019375,70.818058
6,6,0.001014,0.019158,70.818058
7,7,0.000547,0.018974,70.818058
8,8,0.000521,0.018814,70.818058
9,9,0.000616,0.018642,70.818058


In [9]:
# Training
epochs_list = [7, 5, 3, 2, 2]
lrs = [1e-3, 3e-4, 1e-4, 3e-5, 1e-5]
train_losses = []
val_losses = []
for ii, epochs in enumerate(epochs_list):
    print('Setting learning rate to %.1e' % lrs[ii])
    opt = optim.Adam(model_big.parameters(), lr=lrs[ii])
    epoch_train_losses, epoch_val_losses = fit(epochs, model_big, loss_func, opt, train_dl, valid_dl, device)
    train_losses = train_losses + epoch_train_losses
    val_losses = val_losses + epoch_val_losses

Setting learning rate to 1.0e-03


KeyboardInterrupt: 

In [ ]:
plt.plot(train_losses[1:], marker='>', label='training loss')
plt.plot(val_losses[1:], marker='>', label='valid_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
# saving the model for later inference (if training is to be continued another saving method is recommended)
# save_path = './models/AE_big_model_loss48eneg6.pt'
# torch.save(model_big.state_dict(), save_path)
# model_big = AE_big()
# model_big.load_state_dict(torch.load(save_path))
# model_big.eval()

## Evaluation

In [ ]:
model_big.eval()

In [ ]:
# Print a few tensors, still normalized
print('Comparing input and output:')
for ii in np.arange(100, 105):
    data = valid_ds.tensors[0][ii]
    pred = model_big(data)
    print('Inp:', data)
    print('Out:', pred)
    print(' ')

In [ ]:
# Print a few tensors, now not normalized
print('Comparing input and output:')
for ii in np.arange(100, 105):
    data = valid_ds.tensors[0][ii]
    pred = model_big(data).detach().numpy()
    pred = np.multiply(pred, train_std.values)
#    pred = np.multiply(pred, train_max.values)
    pred = np.add(pred, train_mean.values)
    data = np.multiply(data, train_std.values)
#    data = np.multiply(data, train_max.values)
    data = np.add(data, train_mean.values)
    print('Inp:', data)
    print('Out:', pred)
    print(' ')

In [ ]:
plt.close('all')
unit_list = ['[GeV]', '[rad]', '[rad]', '[GeV]']
variable_list = [r'$p_T$', r'$\eta$', r'$\phi$', r'$E$']
line_style = ['--', '-']
colors = ['orange', 'c']
markers = ['*', 's']


# Histograms
idxs = (0, 100000)  # Choose events to compare
data = torch.tensor(test_x[idxs[0]:idxs[1]].values)
pred = model_big(data).detach().numpy()
pred = np.multiply(pred, train_std.values)
#pred = np.multiply(pred, train_max.values)
pred = np.add(pred, train_mean.values)
data = np.multiply(data, train_std.values)
#data = np.multiply(data, train_max.values)
data = np.add(data, train_mean.values)

alph = 0.8
n_bins = 50
for kk in np.arange(4):
    plt.figure(kk + 4)
    n_hist_data, bin_edges, _ = plt.hist(data[:, kk], color=colors[1], label='Input', alpha=1, bins=n_bins)
    n_hist_pred, _, _ = plt.hist(pred[:, kk], color=colors[0], label='Output', alpha=alph, bins=bin_edges)
    plt.suptitle(train.columns[kk])
    plt.xlabel(variable_list[kk] + ' ' + unit_list[kk])
    plt.ylabel('Number of events')
    ms.sciy()

In [ ]:
# Plot input on top of output
idxs = (0, 100)  # Choose events to compare
data = torch.tensor(test_x[idxs[0]:idxs[1]].values)
pred = model_big(data).detach().numpy()
pred = np.multiply(pred, train_std.values)
#pred = np.multiply(pred, train_max.values)
pred = np.add(pred, train_mean.values)
data = np.multiply(data, train_std.values)
#data = np.multiply(data, train_max.values)
data = np.add(data, train_mean.values)

for kk in np.arange(4):
    plt.figure(kk)
    plt.plot(data[:, kk], color=colors[1], label='Input', linestyle=line_style[1], marker=markers[1])
    plt.plot(pred[:, kk], color=colors[0], label='Output', linestyle=line_style[0], marker=markers[0])
    plt.suptitle(train.columns[kk])
    plt.xlabel('Event')
    plt.ylabel(variable_list[kk] + ' ' + unit_list[kk])
    plt.legend()
    ms.sciy()